In [1]:
#FIUBA - 75.26 Simulacion
#2C 2019 - Trabajo final
#79979 - Gonzalez, Juan Manuel (juanmg0511@gmail.com)

#A pseudo-random numbers generator based on a novel 3D chaotic map with an application to color image encryption
#https://doi.org/10.1007/s11071-018-4390-z
#Resistencia a ataques de oclusión

#Importación modulos y librerias
import sys, os, math
from PIL import Image

#SECCION DE CONFIGURACION
#Path de los archivos originales
path_original_a = "../images/lena_std_encrypted_occ_a.tif"
path_original_c = "../images/lena_std_encrypted_occ_c.tif"
path_original_e = "../images/lena_std_encrypted_occ_e.tif"
path_original_g = "../images/lena_std_encrypted_occ_g.tif"
path_original_i = "../images/lena_std_encrypted_occ_i.tif"
path_original_k = "../images/lena_std_encrypted_occ_k.tif"
path_original_m = "../images/lena_std_encrypted_occ_m.tif"
path_original_o = "../images/lena_std_encrypted_occ_o.tif"

#Clave a utilizar
K = (0.4110000047244551, 0.3210000025965682, 0.6310000027632665)

#Definición de parámetros de control: c1, c2, reales
c1 = 20
c2 = 20
#Cantidad de iteraciones y precisión
nt = 10**5
d = 8

#Implementación de funciones auxiliares
#Picewise map
def pMap(x):
    "piecewise map: funcion que dado x y un parametro de control real c1, calcula Ψ(x)"
    return (abs(1 - (c1 * x)))

#Logistic map
def lMap(x, y):
    "2D logistic map: funcion que dados x e y, y un parametro de control real c2, calcula Λ(x, y)"
    return (c2 * x * (1 - y))

#Implemantación del 3D piecewise-logistic map (3D-PLM), T
def tMap(x, y, z):
    "3D piecewise-logistic map: funcion que dados x, y, z, los paráetros de control reales c1 y c2, calcula T(x, y, z)"    
    x = (pMap(x) + lMap(y, z)) % 1
    y = (pMap(y) + lMap(z, x)) % 1
    z = (pMap(z) + lMap(x, y)) % 1
    
    return x, y, z

#Función de discretización Φ
def phi(amin, amax, d, u):
    "Función de discretización Φ:"
    "Φd : [amin, amax] -> [0, ... , d] ; para amin=0 y amax=1"
    "Dados amin, amax y d, calcula: u -> Φd(u)"
    
    h = ((abs(amax-amin))/(d+1))
    
    r = 0
    if (u >= amin and u < amax):   
        r = math.floor((u-amin)/h)    
    else:
        r = d

    return (r)

#Función para generar las mascaras caóticas aplicadas por el algoritmo de encriptación
#Versión optimizada
def generateChaoticMasksV2(K, m, n, nt):
    "Genera las mascaras caoticas utilizando el 3D-PLM"
    "Entrada:"
    "K: la clave de encriptacion"
    "m,n: las dimensiones de la imagen, alto y ancho respectivamente"
    "nt: parametro de transicion del 3D-PLM"
    "Salida:"
    "Las mascaras Mr, Mg, Mb"

    Cx = []
    Cy = []
    Cz = []

    x = K[0]
    y = K[1]
    z = K[2]
    for i in range(nt):
        x, y, z = tMap(x, y, z)
    
    #t = TicToc()
    #t.tic()
    for i in range(m*n):
        if (i==0):
            Cx.append(x)
            Cy.append(y)
            Cz.append(z)
        else:
            x, y, z = tMap(Cx[i-1], Cy[i-1], Cz[i-1])
            Cx.append(x)
            Cy.append(y)
            Cz.append(z)
        #barraProgreso("Generating the chaotic masks",i,(m*n))
        #t.toc()
    #barraProgreso("Done",(m*n),(m*n))
    #t.toc()

    Mr = []
    Mg = []
    Mb = []

    for i in range(m*n):
        Mr.append(phi(0,1,(m*n),Cx[i]))
        Mg.append(phi(0,1,(m*n),Cy[i]))
        Mb.append(phi(0,1,(m*n),Cz[i]))

    return Mr, Mg, Mb

def decrypt(path_encriptada, key):
    #Des-encriptación
    #Apertura de la imagen
    im_encriptada_sc = Image.open(path_encriptada)
    Ie = list(im_encriptada_sc.getdata())

    n = im_encriptada_sc.width
    m = im_encriptada_sc.height

    #Paso 1
    #Preliminary stage
    Cpr = []
    Cpg = []
    Cpb = []

    Cpr = [r[0] for r in Ie]
    Cpg = [g[1] for g in Ie]
    Cpb = [b[2] for b in Ie]

    #Paso 2
    #Generate the chaotic masks
    Mr, Mg, Mb = generateChaoticMasksV2(key,m,n,nt)

    #Paso 3
    #Obtain the shuffled image
    Sr = []
    Sg = []
    Sb = []

    for i in range(m*n):
        Sr.append((Cpr[i] - Mr[i]) % 256)
        Sg.append((Cpg[i] - Mg[i]) % 256)
        Sb.append((Cpb[i] - Mb[i]) % 256)

    #Paso 4
    #Retrieve the original image
    x = key[0]
    y = key[1]
    z = key[2]

    kr = 0
    kg = 0
    kb = 0

    Ir = []
    Ig = []
    Ib = []

    while (kr < n*m or kg < n*m or kb < n*m):

        x, y, z = tMap(x, y, z)

        if ((Sr[phi(0,1,(m*n)-1,x)]) >= 0):
            Ir.append(Sr[phi(0,1,(m*n)-1,x)])
            Sr[phi(0,1,(m*n)-1,x)] = -1
            kr = kr + 1
    
        if ((Sg[phi(0,1,(m*n)-1,y)]) >= 0):
            Ig.append(Sg[phi(0,1,(m*n)-1,y)])
            Sg[phi(0,1,(m*n)-1,y)] = -1
            kg = kg + 1
        
        if ((Sb[phi(0,1,(m*n)-1,z)]) >= 0):
            Ib.append(Sb[phi(0,1,(m*n)-1,z)])
            Sb[phi(0,1,(m*n)-1,z)] = -1
            kb = kb + 1
        
    Id = []
    for i in range(m*n):
        Id.append((Ir[i], Ig[i], Ib[i]))
        
    #Determinación de path con nombre y extensión
    path_encriptada_nombre, path_encriptada_extension = os.path.splitext(path_encriptada)
    
    #Escritura del resultado
    im_desencriptada = Image.new(im_encriptada_sc.mode, im_encriptada_sc.size)
    im_desencriptada.putdata(Id)
    im_desencriptada.save(path_encriptada_nombre + "_decrypted" + path_encriptada_extension,format=im_encriptada_sc.format,quality=100,subsampling=0)
    
    print("Decrypted file \"" + path_encriptada_nombre + "_decrypted" + path_encriptada_extension + "\", " + str(os.stat(path_encriptada_nombre + "_decrypted" + path_encriptada_extension).st_size) + " bytes written.")
    return

In [2]:
#Desencriptamos las imagenes
decrypt(path_original_a, K)
decrypt(path_original_c, K)
decrypt(path_original_e, K)
decrypt(path_original_g, K)
decrypt(path_original_i, K)
decrypt(path_original_k, K)
decrypt(path_original_m, K)
decrypt(path_original_o, K)

Decrypted file "../images/lena_std_encrypted_occ_a_decrypted.tif", 786572 bytes written.
Decrypted file "../images/lena_std_encrypted_occ_c_decrypted.tif", 786572 bytes written.
Decrypted file "../images/lena_std_encrypted_occ_e_decrypted.tif", 786572 bytes written.
Decrypted file "../images/lena_std_encrypted_occ_g_decrypted.tif", 786572 bytes written.
Decrypted file "../images/lena_std_encrypted_occ_i_decrypted.tif", 786572 bytes written.
Decrypted file "../images/lena_std_encrypted_occ_k_decrypted.tif", 786572 bytes written.
Decrypted file "../images/lena_std_encrypted_occ_m_decrypted.tif", 786572 bytes written.
Decrypted file "../images/lena_std_encrypted_occ_o_decrypted.tif", 786572 bytes written.
